In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.spatial.transform import Rotation

%matplotlib qt
# %matplotlib inline


def Rz(phi: float):
    Rz = np.array(
        [[np.cos(phi), -np.sin(phi), 0], [np.sin(phi), np.cos(phi), 0], [0, 0, 1]]
    )
    return Rz


def sph2cart(azimuth, elevation):
    return np.array(
        [
            np.cos(azimuth) * np.cos(elevation),
            np.sin(azimuth) * np.cos(elevation),
            np.sin(elevation),
        ]
    )


def quatFromVec(u, v):
    u = u / np.linalg.norm(u)
    v = v / np.linalg.norm(v)

    if (u == -v).all():
        print("ERRORRR")

    half = (u + v) / np.linalg.norm(u + v)
    w = np.dot(u, half)
    xyz = np.cross(u, half)

    q = np.array([w, *xyz])

    return q / np.linalg.norm(q)


def rotFromQuat(q):
    a, b, c, d = q
    a2, b2, c2, d2 = a * a, b * b, c * c, d * d

    return np.array(
        [
            [a2 + b2 - c2 - d2, 2 * b * c - 2 * a * d, 2 * b * d + 2 * a * c],
            [2 * b * c + 2 * a * d, a2 - b2 + c2 - d2, 2 * c * d - 2 * a * b],
            [2 * b * d - 2 * a * c, 2 * c * d + 2 * a * b, a2 - b2 - c2 + d2],
        ]
    )


np.random.seed(42)

# prop coords
angles = [np.pi / 4, 3 * np.pi / 4, -3 * np.pi / 4, -np.pi / 4]
rotor_length = 1.0
sensor_length = 0.5
sensor_vert = 0  # -0.25

# init plots
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(projection="3d")


def animate(i):
    ax.clear()
    ax.grid(True)
    ax.set_xlabel("$x$ [m]")
    ax.set_ylabel("$y$ [m]")
    ax.set_zlabel("$z$ [m]")
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])

    # draw reference frame
    ax.quiver(0, 0, 0, 1, 0, 0, color="gray", length=0.3)
    ax.quiver(0, 0, 0, 0, 1, 0, color="gray", length=0.3)
    ax.quiver(0, 0, 0, 0, 0, 1, color="gray", length=0.3)

    # np.random.seed(42)
    # deflection = np.random.normal(size=(2,))
    # deflection = [azimuths[i], 0]
    # deflection = [0, elevations[i]]
    deflection = [azimuths[i], elevations[i]]
    # draw quadrotor
    for psi in angles:
        O_t_OR = Rz(psi) @ np.array([rotor_length, 0, 0])
        O_t_OS = Rz(psi) @ np.array([sensor_length, 0, sensor_vert])
        O_rot_S = Rz(np.arctan2(O_t_OS[1], O_t_OS[0]))

        # sensor-relative rotor position
        O_t_SR = O_t_OR - O_t_OS
        S_t_SR = O_rot_S.T @ O_t_SR

        # rotation representing az/el transformation
        q = quatFromVec(np.array([1, 0, 0]), sph2cart(deflection[0], deflection[1]))
        rotm = rotFromQuat(q)

        # frame conversions
        S_t_SRd = rotm @ S_t_SR
        O_t_SRd = O_rot_S @ S_t_SRd
        r = O_t_SRd + O_t_OS

        f_nom = 0.5 * np.array([0, 0, 1])
        # REVIEW: maybe doesn't make sense
        f = O_rot_S @ rotm @ f_nom

        # rigid quad
        ax.quiver(0, 0, 0, *O_t_OR, color="k", alpha=0.25, label="Nominal Quad")
        ax.quiver(*O_t_OR, *f_nom, color="r", alpha=0.25)
        # sensor
        ax.quiver(0, 0, 0, *O_t_OS, color="b", label="Pre-Joint")
        # ax.quiver(*O_t_OS, *O_t_SR, color="g")
        # deflections
        ax.quiver(*O_t_OS, *O_t_SRd, color="g", label="Post-Joint")
        ax.quiver(*r, *f, color="r", label="Force")
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())


N = 100
# azimuths = np.deg2rad(np.linspace(-30, 30, N))
# elevations = np.deg2rad(np.linspace(-30, 30, N))
steps = N // 5
azimuths = np.deg2rad(
    [
        *np.linspace(0, -30, steps),
        *np.linspace(-30, 30, steps),
        *np.linspace(30, 30, steps),
        *np.linspace(30, 0, steps),
    ]
)
elevations = np.deg2rad(
    [
        *np.linspace(0, -30, steps),
        *np.linspace(-30, -30, steps),
        *np.linspace(-30, 30, steps),
        *np.linspace(30, 0, steps),
    ]
)
ani = animation.FuncAnimation(fig, animate, frames=N, interval=1)

save = False
if save:
    writer = animation.PillowWriter(fps=15, metadata=dict(artist="me"), bitrate=1800)
    ani.save("deflection.gif", writer=writer)